# World Data


# 전처리 시나리오
1. 데이터 읽기
2. 결측값 처리
3. 특수 문자 제거
4. 스케일링
5. 결과 저장

1. 데이터 읽기

In [89]:
import pandas as pd

data = pd.read_csv('world-data.csv')

# 데이터 헤드 출력
print("#"*10)
print('데이터의 헤드')
print(data.head())



##########
데이터의 헤드
       Country Density(P/Km2)  Agricultural Land(%) Land Area(Km2)  \
0  Afghanistan             60                  58.1        652,230   
1      Albania            105                  43.1         28,748   
2      Algeria             18                  17.4      2,381,741   
3      Andorra            164                  40.0            468   
4       Angola             26                  47.5      1,246,700   

  Armed Forces size  Birth Rate  Calling Code Capital/Major City  \
0           323,000       32.49          93.0              Kabul   
1             9,000       11.78         355.0             Tirana   
2           317,000       24.28         213.0            Algiers   
3               NaN        7.20         376.0   Andorra la Vella   
4           117,000       40.73         244.0             Luanda   

  Co2-Emissions     CPI  CPI Change(%)  
0         8,672   149.9            2.3  
1         4,536  119.05            1.4  
2       150,006  151.36     

In [90]:
# 데이터의 칼럼 정보 출력
print("#"*10)
print('데이터의 각 칼럼에 대한 정보')
print(data.info())

##########
데이터의 각 칼럼에 대한 정보
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Country               195 non-null    object 
 1   Density(P/Km2)        195 non-null    object 
 2   Agricultural Land(%)  188 non-null    float64
 3   Land Area(Km2)        194 non-null    object 
 4   Armed Forces size     171 non-null    object 
 5   Birth Rate            189 non-null    float64
 6   Calling Code          194 non-null    float64
 7   Capital/Major City    192 non-null    object 
 8   Co2-Emissions         188 non-null    object 
 9   CPI                   178 non-null    object 
 10  CPI Change(%)         179 non-null    float64
dtypes: float64(4), object(7)
memory usage: 16.9+ KB
None


In [91]:
# 데이터 기초 통계 확인
data.describe()

,Agricultural Land(%),Birth Rate,Calling Code,CPI Change(%)
count,188.000000,189.000000,194.000000,179.000000
mean,39.117553,20.214974,360.546392,6.722346
std,21.783052,9.945774,323.236419,24.450414
min,0.600000,5.900000,1.000000,-4.300000
25%,21.700000,11.300000,82.500000,1.000000
50%,39.600000,17.950000,255.500000,2.300000
75%,55.375000,28.750000,506.750000,4.250000
max,82.600000,46.080000,1876.000000,254.900000


In [92]:
data.head()

,Country,Density(P/Km2),Agricultural Land(%),Land Area(Km2),Armed Forces size,Birth Rate,Calling Code,Capital/Major City,Co2-Emissions,CPI,CPI Change(%)
0,Afghanistan,60,58.1,"652,230","323,000",32.49,93.0,Kabul,"8,672",149.9,2.3
1,Albania,105,43.1,"28,748","9,000",11.78,355.0,Tirana,"4,536",119.05,1.4
2,Algeria,18,17.4,"2,381,741","317,000",24.28,213.0,Algiers,"150,006",151.36,2.0
3,Andorra,164,40.0,468,NaN,7.20,376.0,Andorra la Vella,469,NaN,NaN
4,Angola,26,47.5,"1,246,700","117,000",40.73,244.0,Luanda,"34,693",261.73,17.1


In [93]:
data.tail()

,Country,Density(P/Km2),Agricultural Land(%),Land Area(Km2),Armed Forces size,Birth Rate,Calling Code,Capital/Major City,Co2-Emissions,CPI,CPI Change(%)
190,Venezuela,32,24.5,"912,050","343,000",17.88,58.0,Caracas,"164,175","2,740.27",254.9
191,Vietnam,314,39.3,"331,210","522,000",16.75,84.0,Hanoi,"192,668",163.52,2.8
192,Yemen,56,44.6,"527,968","40,000",30.45,967.0,Sanaa,"10,609",157.58,8.1
193,Zambia,25,32.1,"752,618","16,000",36.19,260.0,Lusaka,"5,141",212.31,9.2
194,Zimbabwe,38,41.9,"390,757","51,000",30.68,263.0,Harare,"10,983",105.51,0.9


2. 결측치 확인

In [94]:
# 결측값 확인
missing_values = data.isnull().sum()
print('Missing values in each column:\n', missing_values)

Missing values in each column:
 Country                  0
Density(P/Km2)           0
Agricultural Land(%)     7
Land Area(Km2)           1
Armed Forces size       24
Birth Rate               6
Calling Code             1
Capital/Major City       3
Co2-Emissions            7
CPI                     17
CPI Change(%)           16
dtype: int64


### 처리해야 되는 결측치 columns
1. Agricultural Land(농지): 해당 국가의 평균값 또는 중앙값으로 대체하거나 인근 국가의 유사한 비율로 대체
2. Land area(면적): 결측치가 1이기 때문에 통계자료로 작성 또는 평균값으로 대체
3. armed Forces size(군대 규모): 사용 안함(분석에 필요 없음)
4. Birth Rate(출생률): 평균값으로 대체하거나 사용 안함
5. Calling code: 국가별 전화 코드는 다르기 때문에 사용하지 않거나 찾아서 등록(분석에 필요 없음)
6. Capital/Major City(수도/주요 도시):각 국가의 수도 또는 주요 도시를 수동으로 찾아서 결측값을 대체(분석에 필요 없음)
7. Co2-Emissions(Co2 배출량):평균값 또는 중앙값 사용 데이터의 중심 경향을 유지
8. CPI(소비자 물가 지수): 평균값 또는 중앙값 사용 데이터의 중심 경향을 유지
9. CPI Change(CPI 변화율): 연속형 변수이므로 평균값을 사용하여 결측값 대체 또는 이상치의 영향을 줄이기 위해 중앙값 사용

3. 특수 문자 제거(csv 데이터들중 수치형 데이터가 object형으로 적용되어 있음) 

In [95]:
# 쉼표를 제거하고 숫자로 변환하는 함수
def remove_comma_and_convert(series):
    return series.str.replace(',', '').astype(float)

# 쉼표를 포함한 열 변환
for column in ['Density(P/Km2)','Land Area(Km2)', 'Armed Forces size','Co2-Emissions','CPI']:
    if column in data.columns:
        data[column] = remove_comma_and_convert(data[column])




In [102]:
# 문자 제거 후 데이터 타입 확인
print(f"문자 제거 후 데이터 타입:\n{data.dtypes}")

문자 제거 후 데이터 타입:
Country                  object
Density(P/Km2)          float64
Agricultural Land(%)    float64
Land Area(Km2)          float64
Armed Forces size       float64
Birth Rate              float64
Calling Code            float64
Capital/Major City       object
Co2-Emissions           float64
CPI                     float64
CPI Change(%)           float64
dtype: object


4. 결츨치 처리 및 스케일링(파이프 라인)

In [152]:
# 전처리 작업의 자동화 (파이프라인 구축)
from sklearn.pipeline import Pipeline # 파이프라인 구축을 위한 클래스
from sklearn.impute import SimpleImputer  # 결측값 처리
from sklearn.preprocessing import StandardScaler  # 데이터 스케일링
import numpy as np

# 데이터 분리
data_categorical= data[['Country']]
data_numerical=data[['Density(P/Km2)','Land Area(Km2)', 'Birth Rate', 'Co2-Emissions', 'CPI', 'CPI Change(%)']]
# 결측값 대체: 평균값으로 대체
mean_imputer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
])


# 데이터 변환
processed_numerical = mean_imputer.fit_transform(data_numerical)

# 결과를 데이터프레임으로 변환
processed_numerical_df = pd.DataFrame(processed_numerical, columns=data_numerical.columns)

# 최종 데이터프레임 결합
final_data = pd.concat([data_categorical,processed_numerical_df], axis=1)

print("\nProcessed Data:")
print(final_data)


Processed Data:
         Country  Density(P/Km2)  Land Area(Km2)  Birth Rate  Co2-Emissions  \
0    Afghanistan            60.0        652230.0       32.49         8672.0   
1        Albania           105.0         28748.0       11.78         4536.0   
2        Algeria            18.0       2381741.0       24.28       150006.0   
3        Andorra           164.0           468.0        7.20          469.0   
4         Angola            26.0       1246700.0       40.73        34693.0   
..           ...             ...             ...         ...            ...   
190    Venezuela            32.0        912050.0       17.88       164175.0   
191      Vietnam           314.0        331210.0       16.75       192668.0   
192        Yemen            56.0        527968.0       30.45        10609.0   
193       Zambia            25.0        752618.0       36.19         5141.0   
194     Zimbabwe            38.0        390757.0       30.68        10983.0   

             CPI  CPI Change(%)  


In [153]:
#최종 데이터의 결측치 값
print('Missing values in each column:\n')
final_data.isnull().sum()

Missing values in each column:



Country           0
Density(P/Km2)    0
Land Area(Km2)    0
Birth Rate        0
Co2-Emissions     0
CPI               0
CPI Change(%)     0
dtype: int64

In [154]:

print(f"Final data columns dtypes: \n{final_data.dtypes}")

Final data columns dtypes: 
Country            object
Density(P/Km2)    float64
Land Area(Km2)    float64
Birth Rate        float64
Co2-Emissions     float64
CPI               float64
CPI Change(%)     float64
dtype: object


In [155]:
# 전처리 전후 데이터
print(f"Original Data: \n{data}")
print(f"Final Data: \n{final_data}")

Original Data: 
         Country  Density(P/Km2)  Agricultural Land(%)  Land Area(Km2)  \
0    Afghanistan            60.0                  58.1        652230.0   
1        Albania           105.0                  43.1         28748.0   
2        Algeria            18.0                  17.4       2381741.0   
3        Andorra           164.0                  40.0           468.0   
4         Angola            26.0                  47.5       1246700.0   
..           ...             ...                   ...             ...   
190    Venezuela            32.0                  24.5        912050.0   
191      Vietnam           314.0                  39.3        331210.0   
192        Yemen            56.0                  44.6        527968.0   
193       Zambia            25.0                  32.1        752618.0   
194     Zimbabwe            38.0                  41.9        390757.0   

     Armed Forces size  Birth Rate  Calling Code Capital/Major City  \
0             323000.0  

데이터셋이 195x7 이므로 작은 데이터에 포함되어 샘플링 필요치 않음
위의 데이터셋은 7개의 변수로 구성되어 있으며, 고차원 데이터로 분류되지 않아 PCA, t-SNE 필요없음

# 데이터 분석 시나리오
1. 데이터의 종류와 속성
    - 주어진 데이터셋에서 범주형 및 연속형 데이터 열을 식별하고, 각 열의 기술 통계와 빈도수를 출력
2. 데이터 탐색(EDA)
    - 제공된 데이터셋의 결측치와 중복을 파악하고, 적절한 처리 방안을 제안
3. 기초통계
    - 데이터셋의 왜도와 첨도를 계산하고, 그 의미에 대해 설명
4. 상관관계와 인과관계
    - 선택한 두 변수 간의 상관관계를 계산하고, 그 결과를 해석, 이를 바탕으로 가능한 인과관계 논의
5. 가설검정과 A/B 테스트
    - 주어진 데이터를 사용하여 A/B 테스트를 설계하고, 가설을 설정한 후 통계적 검정을 수행. 결과 해설 및 결론 도출
    - 본인의 수집한 데이터에서 가설 검정, A/B 테스트를 수행해보기 어렵다면 어떤 가설을 설정하고 A/B 테스트를 어떻게 설계했을지 생각

### 1. 데이터의 종류와 속성

In [157]:
#데이터 타입 확인
print('데이터 타입 확인')
print('Data Types:\n', final_data.dtypes)

#범주형 및 수치형 데이터 분리하여 분석
categorical_cols = final_data.select_dtypes(include = ['object','category']).columns
numerical_cols = final_data.select_dtypes(include=['int64','float64']).columns

print('#'*10)
print('범주형 데이터 분리하여 분석')
print("\nCategorical Columns: \n", categorical_cols)

print('#'* 10)
print('수치형 데이터 분리하여 분석')
print("\nNumertical Columns:\n", numerical_cols)

데이터 타입 확인
Data Types:
 Country            object
Density(P/Km2)    float64
Land Area(Km2)    float64
Birth Rate        float64
Co2-Emissions     float64
CPI               float64
CPI Change(%)     float64
dtype: object
##########
범주형 데이터 분리하여 분석

Categorical Columns: 
 Index(['Country'], dtype='object')
##########
수치형 데이터 분리하여 분석

Numertical Columns:
 Index(['Density(P/Km2)', 'Land Area(Km2)', 'Birth Rate', 'Co2-Emissions',
       'CPI', 'CPI Change(%)'],
      dtype='object')


### 2. 데이터 탐색

In [158]:
# 결측치 확인
print("\nMissing Calues:\n", final_data.isnull().sum())

# 각 범주형 변수의 유니크한 값과 빈도수
for col in categorical_cols:
    print(f"\n Unique calues in {col}:\n", final_data[col].value_counts())
    
# 수치형 데이터의 기초 통계
print("\nDescriptive Statistics for Numerical Data:\n", final_data[numerical_cols].describe())


Missing Calues:
 Country           0
Density(P/Km2)    0
Land Area(Km2)    0
Birth Rate        0
Co2-Emissions     0
CPI               0
CPI Change(%)     0
dtype: int64

 Unique calues in Country:
 Country
Afghanistan      1
Saint Lucia      1
Nicaragua        1
Niger            1
Nigeria          1
                ..
Grenada          1
Guatemala        1
Guinea           1
Guinea-Bissau    1
Zimbabwe         1
Name: count, Length: 195, dtype: int64

Descriptive Statistics for Numerical Data:
        Density(P/Km2)  Land Area(Km2)  Birth Rate  Co2-Emissions          CPI  \
count      195.000000    1.950000e+02  195.000000   1.950000e+02   195.000000   
mean       356.764103    6.896244e+05   20.214974   1.777992e+05   190.460955   
std       1982.888967    1.916650e+06    9.790765   8.235184e+05   380.111858   
min          2.000000    0.000000e+00    5.900000   1.100000e+01    99.030000   
25%         35.500000    2.445650e+04   11.450000   2.625500e+03   114.470000   
50%         8

### 3. 기초 통계

In [159]:
# 왜도와 첨도 확인
"""
왜도(Skewness): 0에 가까울수록 정규분포에 근사, 양의 값은 오른쪽 꼬리가 긴 분포(왼쪽으로 치우친), 음의 값은 왼쪽 꼬리가 긴 분포(오른쪽으로 치우친)
첨도(Kurtosis): 0에 가까울수록 정규분포에 근사, 높으면 분포가 뾰족하고, 낮으면 평평
"""
print("\nSkewness of the data:\n", final_data[numerical_cols].skew())
print("\nKurtosis of the data:\n", final_data[numerical_cols].kurt())


Skewness of the data:
 Density(P/Km2)    12.064621
Land Area(Km2)     5.583425
Birth Rate         0.587640
Co2-Emissions      9.747288
CPI                9.759931
CPI Change(%)      8.716789
dtype: float64

Kurtosis of the data:
 Density(P/Km2)    155.050367
Land Area(Km2)     36.191264
Birth Rate         -0.750688
Co2-Emissions     106.583337
CPI               103.045134
CPI Change(%)      83.236752
dtype: float64


왜도와 첨도 호가인 결과 이상치가 많고 비대칭적인 분포를 가짐<br>
이를 해결하기 위해서는 이상치를 탐지하고 제거해야함

### 4. 상관관계와 인과관계

In [161]:
"""
상관계수 값이 1에 가까울수록 완벽한 양의 상관관계, -1에 가까울수록 완벽한 음의 상관관계를 나타냅니다.
"""

# 피어슨 상관 계수
print("Pearson Correlation:\n", final_data[numerical_cols].corr(method='pearson'))

# 스피어만 상관 계수
print("\nSpearman Correlation:\n", final_data[numerical_cols].corr(method='spearman'))

Pearson Correlation:
                 Density(P/Km2)  Land Area(Km2)  Birth Rate  Co2-Emissions  \
Density(P/Km2)        1.000000       -0.053270   -0.146719      -0.004632   
Land Area(Km2)       -0.053270        1.000000   -0.066158       0.589550   
Birth Rate           -0.146719       -0.066158    1.000000      -0.154123   
Co2-Emissions        -0.004632        0.589550   -0.154123       1.000000   
CPI                  -0.014074        0.020806    0.137816      -0.017445   
CPI Change(%)        -0.017032        0.034073    0.103383      -0.004196   

                     CPI  CPI Change(%)  
Density(P/Km2) -0.014074      -0.017032  
Land Area(Km2)  0.020806       0.034073  
Birth Rate      0.137816       0.103383  
Co2-Emissions  -0.017445      -0.004196  
CPI             1.000000       0.894704  
CPI Change(%)   0.894704       1.000000  

Spearman Correlation:
                 Density(P/Km2)  Land Area(Km2)  Birth Rate  Co2-Emissions  \
Density(P/Km2)        1.000000       -0.565

#### 상관관계)
- 강한 상관관계: 
    - CPI, CPI Change -> 매우 강한 양의 상관관계로 소비자 물가 지수와 그 변화율은 매우 밀접한 관련이 있음
    - Land Area와 Co2-Emissings -> 중간 정도의 양의 상관관계로 토지 면적이 클수록 CO2 배출량도 많아지는 경향이 있음
- 약한 상관관계:
    - Birth Rate와 CPI -> 출생률과 소비자 물가 지수 간에 약한 양의 상관관계가 있음
    - Birth Rate와 Co2-Emissions -> 출생률과 CO2 배출햘 간에 약한 음의 상관관계를 가지고 있음

#### 인과관계)
- 이를 바탕으로 소비자 물가 지수가 높아지면 소비자 물가 지수 변화율도 높아질 가능성이 있습니다. 이 때문에 두 변수 간에 인과관계가 있을 가능성을 시사함

### 가설 설정
- 목적
    - 소비자 물가 지수(CPI)가 소비자 물가 지수 변화율(CPI Change%)에 미치는 영향을 평가하기 위해 A/B 테스트 설계

- 가설
    - 귀무 가설(H0): CPI가 높아지더라도 CPI Change%에는 유의미한 변화가 없다
    - 대립 가설(H1): CPI가 높아지면 CPI Change%가 유의미하게 변화한다

- A/B 테스트 설계
    1. 그룹 나누기
        - 데이터를 두 그룹으로 나눕니다: 낮은 CPI 그룹과 높은 CPI 그룹
        - CPI의 중앙값을 기준으로 그룹을 나눕니다
    2. 데이터 수집
        - 주어진 데이터셋에서 CPI와 CPI Change% 값을 추출합니다
    3. 통계적 검정
        - 두 그룹의 CPI Change% 평균을 비교하여 유의마한 차이가 있는지 검증
        - t-검정(T-test)을 사용하여 두 그룹 간의 평균 차이를 평가

In [162]:
from scipy import stats

"""
T-통계량의 절대값이 크면 클수록 두 그룹 간의 차이가 크다고 할 수 있습니다.

일반적으로 P-값이 0.05보다 작으면 귀무 가설을 기각하고, 통계적으로 유의미한 차이가 있음을 인정합니다
"""

df = pd.DataFrame(final_data)

# CPI의 중위값을 기준으로 그룹 나누기
median_cpi = df['CPI'].median()
group_low_cpi = df[df['CPI'] <= median_cpi]['CPI Change(%)']
group_high_cpi = df[df['CPI'] > median_cpi]['CPI Change(%)']

# 두 그룹의 평균 비교
mean_low_cpi = group_low_cpi.mean()
mean_high_cpi = group_high_cpi.mean()

print(f"낮은 CPI 그룹의 평균 CPI Change(%): {mean_low_cpi}")
print(f"높은 CPI 그룹의 평균 CPI Change(%): {mean_high_cpi}")

# t-검정
t_stat, p_value = stats.ttest_ind(group_low_cpi, group_high_cpi)
print(f"t-statistic: {t_stat}, p-value: {p_value}")

# 결과 해석
alpha = 0.05
if p_value < alpha:
    print("유의수준 0.05에서 귀무 가설을 기각합니다. 높은 CPI가 CPI Change%에 유의미한 영향을 미칩니다.")
else:
    print("유의수준 0.05에서 귀무 가설을 기각할 수 없습니다. 높은 CPI가 CPI Change%에 유의미한 영향을 미친다고 할 수 없습니다.")

낮은 CPI 그룹의 평균 CPI Change(%): 1.2020408163265306
높은 CPI 그룹의 평균 CPI Change(%): 12.299562287623106
t-statistic: -3.3970375906754566, p-value: 0.0008272060655264432
유의수준 0.05에서 귀무 가설을 기각합니다. 높은 CPI가 CPI Change%에 유의미한 영향을 미칩니다.
